In [ ]:
%aiida
from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

In [ ]:
from aiida.orm.data.structure import StructureData
from aiida.orm.data.parameter import ParameterData
from aiida.orm.data.array import ArrayData
from aiida.orm.data.base import Int, Str, Float, Bool
from aiida.orm.data.remote import RemoteData
from aiida.work import workfunction
from aiida.work.process import WorkCalculation
from aiida.work.run import submit
from aiida_cp2k.calculations import Cp2kCalculation
from aiida.orm import Code, Computer
from aiida.orm.querybuilder import QueryBuilder

import ase
import ase.io
import numpy as np
import nglview
from copy import deepcopy
from pprint import pprint

import ipywidgets as ipw
from IPython.display import display, clear_output, HTML

# Select structure

In [ ]:
from apps.surfaces.structure_browser import StructureBrowser

def on_struct_change(c):
    update_view()
    guess_molecule()

    
struct_browser = StructureBrowser()
struct_browser.results.observe(on_struct_change, names='value')
viewer = nglview.NGLWidget()
clear_output()

display(ipw.VBox([struct_browser, viewer]))

def update_view():
    # remove old components
    if hasattr(viewer, "component_0"):
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_unitcell()
        cid = viewer.component_0.id
        viewer.remove_component(cid)
        
    structure = struct_browser.results.value
    if not structure:
        return
    
    atoms = structure.get_ase()
    #print("CELL:", atoms.cell)
    # add new component
    viewer.add_component(nglview.ASEStructure(atoms)) # adds ball+stick
    viewer.add_unitcell()
    viewer.center()
    
    mol_inds = read_mol_indexes() - 1
    
    viewer.add_representation('ball+stick', selection=mol_inds, color='green', aspectRatio=4.0, opacity=0.4)
    

In [ ]:
def get_opt_wfn_file(structure):
    """Tries to find the path to the RESTART.wfn file, if it exists."""
    
    cp2k_calc = structure.get_inputs()[0]
    comp = cp2k_calc.get_computer()
    folder = cp2k_calc.out.remote_folder.get_remote_path()
    
    return comp, folder

# Select molecule

In [ ]:
def guess_molecule():
    try:
        ase_struct = struct_browser.results.value.get_ase()
        first_slab_atom = np.argwhere( (ase_struct.numbers == 29) |
                                       (ase_struct.numbers == 47) |
                                       (ase_struct.numbers == 79)
                                     )[0, 0]
        last_slab_atom = len(ase_struct.numbers)
        mol_indexes_text.value="%d..%d"%(1, first_slab_atom)
    except:
        print("Unable to automatically find slab and molecule")

def read_mol_indexes():
    mol_at_inds = []
    for at_range in mol_indexes_text.value.split():
        ats = at_range.split('..')
        if len(ats) == 1:
            mol_at_inds.append(int(ats[0]))
        elif len(ats) == 2:
            mol_at_inds += list(np.arange(int(ats[0]), int(ats[1])+1))
        else:
            print("Warning: didn't understand mol input.")
    return np.unique(np.array(mol_at_inds))

def get_mol_ase():
    mol_inds = read_mol_indexes()
    ase_struct = struct_browser.results.value.get_ase()
    return ase_struct[read_mol_indexes()-1]
    

style = {'description_width': '120px'}
layout = {'width': '60%'}

mol_indexes_text = ipw.Text(description='Molecule indexes',
                              placeholder='1..10 15..20 25',
                              style=style, layout={'width': '50%'})

highlight_btn = ipw.Button(description='highlight',
                            layout={'width': '10%'})

highlight_btn.on_click(lambda b: update_view())

display(ipw.HBox([mol_indexes_text, highlight_btn]))

# Select computer and codes

In [ ]:
qb = QueryBuilder()
qb.append(Computer, filters={'enabled': True}, project='name')
computer_names = [comp[0] for comp in qb.all()]

style = {'description_width': '120px'}
layout = {'width': '70%'}
drop_computer = ipw.Dropdown(description="Computer",
                             options=computer_names)

def comp_plugin_codes(computer_name, plugin_name):
    qb = QueryBuilder()
    qb.append(Computer, filters={'enabled': True}, project='name', tag='computer')
    qb.append(Code, project='*', has_computer='computer', filters={'attributes.input_plugin': plugin_name})
    qb.order_by({Code: {'id': 'desc'}})
    codes = qb.all()
    sel_codes = []
    for code in codes:
        if code[0] == computer_name:
            sel_codes.append(code[1])
    return sel_codes

def on_computer_change(c):
    global cp2k_codes, eval_morb_codes, stm_image_codes
    cp2k_codes = comp_plugin_codes(drop_computer.value, 'cp2k')
    eval_morb_codes = comp_plugin_codes(drop_computer.value, 'stm.eval_morbs')
    stm_image_codes = comp_plugin_codes(drop_computer.value, 'stm.stm_image')
    
    drop_cp2k.options = [c.label for c in cp2k_codes]
    drop_eval.options = [c.label for c in eval_morb_codes]
    drop_imag.options = [c.label for c in stm_image_codes]
    
    
drop_computer.observe(on_computer_change)

drop_cp2k = ipw.Dropdown(description="Cp2k code")

drop_eval = ipw.Dropdown(description="Eval code")

drop_imag = ipw.Dropdown(description="Image code")

on_computer_change(0)

elpa_check = ipw.Checkbox(
    value=True,
    description='use ELPA',
    disabled=False
)

display(drop_computer, drop_cp2k, drop_eval, drop_imag, elpa_check)

# Orbital overlap parameters

In [ ]:
num_homo_inttext = ipw.IntText(
                    value=4,
                    description="num gas HOMO",
                    style=style, layout=layout_small)
num_lumo_inttext = ipw.IntText(
                    value=4,
                    description="num gas LUMO",
                    style=style, layout=layout_small)

elim_inttext = ipw.IntText(
                    value=4,
                    description="num gas LUMO",
                    style=style, layout=layout_small)

emin_floattext = ipw.BoundedFloatText(
                        description='Slab system Emin (eV)',
                        min=-3.0,
                        max=-0.1,
                        step=0.1,
                        value=-2.0,
                        style=style, layout=layout_small)

emax_floattext = ipw.BoundedFloatText(
                        description='Slab system Emax (eV)',
                        min=0.1,
                        max=3.0,
                        step=0.1,
                        value=2.0,
                        readout_format='%.2f',
                        style=style, layout=layout_small)

display(num_homo_inttext, num_lumo_inttext, emin_floattext, emax_floattext)

In [ ]:
def on_submit(b):
    with submit_out:
        clear_output()
        if not struct_browser.results.value:
            print("Please select a structure.")
            return
        if not drop_computer.value:
            print("Please select a computer.")
            return
        
        bias_voltages = np.array(bias_voltages_text.value.split(), dtype=np.float)
        #height_values = np.array(const_height_text.value.split(), dtype=np.float)
        #current_values = np.array(const_current_text.value.split(), dtype=np.float)

        extrap_plane = extrap_plane_floattext.value
        #num_homo = int(num_homo_text.value)
        #num_lumo = int(num_lumo_text.value)

        #sts_heights = np.array(sts_heights_text.value.split(), dtype=np.float)
        #sts_de = float(sts_de_text.value)
        #sts_fwhm = float(sts_fwhm_text.value)
        
        parent_dir = "parent_calc/"

        eval_orbs_params = ParameterData(dict={
            '--cp2k_input':  parent_dir+'aiida.inp',
            '--basis_file':  parent_dir+'BASIS_MOLOPT',
            '--xyz_file':    parent_dir+'geom.xyz',
            '--wfn_file':    parent_dir+'aiida-RESTART.wfn',
            '--output_file': 'morbs_dx0.2',
            '--emin':        str(np.min(bias_voltages)-0.1),
            '--emax':        str(np.max(bias_voltages)+0.1),
            '--eval_region': ['G', 'G', 'G', 'G', 'b-2.0_C', 't%.1f'%extrap_plane],
            '--dx':          '0.2',
            '--eval_cutoff': '14.0'
        })

        stm_image_params = ParameterData(dict={
            '--npz_file':          parent_dir+'morbs_dx0.2.npz',
            '--hartree_file':      parent_dir+parent_dir+'aiida-HART-v_hartree-1_0.cube',
            '--extrap_plane':      str(extrap_plane),
            '--extrap_extent':     '8.0',
            '--output_dir':        'stm_output',
        #
            '--bias_voltages':     bias_voltages_text.value.split(),
            '--stm_plane_heights': const_height_text.value.split(),
            '--stm_isovalues':     const_current_text.value.split(),
        #
            '--sts_plane_heights': sts_heights_text.value.split(),
            '--sts_de':            str(sts_de_floattext.value),
            '--sts_fwhm':          str(sts_fwhm_floattext.value),
            '--sts_elim':          [str(np.min(bias_voltages)), str(np.max(bias_voltages))],
        #
            '--orb_plane_heights': const_height_text.value.split(),
            '--n_homo':            str(num_homo_inttext.value),
            '--n_lumo':            str(num_lumo_inttext.value),
            '--skip_figs':         ''
        })
        
        cp2k_code = cp2k_codes[drop_cp2k.index]
        eval_orbs_code = eval_morb_codes[drop_eval.index]
        stm_image_code = stm_image_codes[drop_imag.index]
        
        struct = struct_browser.results.value
        
        cell = ArrayData()
        cell.set_array('cell', np.diag(atoms.cell))
        
        ## Try to access the restart-wfn file ##
        try:
            cp2k_calc = struct.get_inputs()[0]
            comp = cp2k_calc.get_computer()
            folder = cp2k_calc.out.remote_folder.get_remote_path()
            if comp.name == drop_computer.value:
                wfn_file_path = folder + "/aiida-RESTART.wfn"
                print("Looking for WFN file on same computer: " + wfn_file_path)
            else:
                wfn_file_path = ""
        except:
            wfn_file_path = ""
    
        
        outputs = submit(STMWorkChain,
                 cp2k_code=cp2k_code,
                 structure=struct,
                 cell=cell,
                 wfn_file_path=Str(wfn_file_path),
                 elpa_switch=Bool(elpa_check.value),
                 eval_orbs_code=eval_orbs_code,
                 eval_orbs_params=eval_orbs_params,
                 stm_image_code=stm_image_code,
                 stm_image_params=stm_image_params
                )
                
        print(outputs)

btn_submit = ipw.Button(description="Submit")
btn_submit.on_click(on_submit)
submit_out = ipw.Output()
display(btn_submit, submit_out)